In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
REPO_NAME = 'sewer-nfl'
import sys, os
cwd = str(os.getcwd())
repo_dir = cwd[:cwd.find(REPO_NAME)+len(REPO_NAME)]
sys.path.insert(0,repo_dir)

In [4]:
import pandas as pd
import nfl_data_py as nflreadr
from warehouse.utilities.elo import calculate_elo_metric

In [5]:
pbp_api_data = nflreadr.import_pbp_data(range(2018,2023))
pbp_api_data = pbp_api_data[pbp_api_data['week']<=18]

2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


In [6]:
roster_api_data = nflreadr.import_rosters(years=[2018,2022])

In [7]:
from warehouse.pipelines.pbp.epa import pre_elo_epa
from warehouse.utilities.elo import calculate_elo_metric

In [19]:
receiving_off_epa , receiving_def_epa = pre_elo_epa(
    pbp_api = pbp_api_data,
    roster_api = roster_api_data, 
    position_filter = ['RB','WR','TE'],
    play_types = ['pass'],
    player_id_col = 'receiver_player_id',
    order_cols = ['season','week'],
    off_gb_cols = ['posteam','defteam','position','player_name'], # At what level are we AGGREGATING offensive performance
    gb_cols_z = ['season','position'], # What is aggregate level for z score
    def_gb_cols = ['posteam','defteam','position'], # Matched up against what level on defense to AGGREGATE to
    perf_cols = ['epa'], # What are we assessing for ELO
    off_appearance_columns = ['position','receiver_player_id'],
    def_appearance_columns = ['position','defteam']
)
receiving_elo_df = calculate_elo_metric(
    input_off_data = receiving_off_epa,
    input_def_data = receiving_def_epa,
    order_cols = ['season','week'],
    off_gb_cols = ['posteam','defteam','position','player_name'], # At what level are we AGGREGATING offensive performance
    def_gb_cols = ['posteam','defteam','position'], # Matched up against what level on defense to AGGREGATE to
    off_appearance_columns = ['position','receiver_player_id'],
    def_appearance_columns = ['position','defteam'],
    off_perf_col = 'z_epa_x',
    def_perf_col = 'z_epa_y',
    elo_multiplier = 5,
    elo_power = 1.7,
    elo_base = 2000
)